In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import imageio
import gc

from tqdm.auto import tqdm

import _pickle as pickle
import tensorflow as tf
def save(file,name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'wb')
    else:
        outfile = open(name+'.pickle', 'wb')
    pickle.dump(file, outfile, protocol=4)
    outfile.close
    
def load(name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'rb')
    else:
        outfile = open(name+'.pickle', 'rb')
    file = pickle.load(outfile)
    outfile.close
    return file

import cv2
from multiprocess import Pool
import tensorflow_hub as hub

import autokeras as ak

In [9]:
import os
import pandas as pd

In [12]:
import autosklearn

ModuleNotFoundError: No module named 'autosklearn'

In [7]:
path

['gs://cassava_gao/raw/test/0/1000015157.jpg',
 'gs://cassava_gao/raw/test/0/102039365.jpg',
 'gs://cassava_gao/raw/test/0/1027796931.jpg',
 'gs://cassava_gao/raw/test/0/1040315063.jpg',
 'gs://cassava_gao/raw/test/0/1072326766.jpg',
 'gs://cassava_gao/raw/test/0/1078590865.jpg',
 'gs://cassava_gao/raw/test/0/1080407872.jpg',
 'gs://cassava_gao/raw/test/0/109816879.jpg',
 'gs://cassava_gao/raw/test/0/1182217128.jpg',
 'gs://cassava_gao/raw/test/0/1185995427.jpg',
 'gs://cassava_gao/raw/test/0/1231451126.jpg',
 'gs://cassava_gao/raw/test/0/1258586775.jpg',
 'gs://cassava_gao/raw/test/0/1281314946.jpg',
 'gs://cassava_gao/raw/test/0/1298789590.jpg',
 'gs://cassava_gao/raw/test/0/1339257499.jpg',
 'gs://cassava_gao/raw/test/0/1342343254.jpg',
 'gs://cassava_gao/raw/test/0/1371699817.jpg',
 'gs://cassava_gao/raw/test/0/1396822256.jpg',
 'gs://cassava_gao/raw/test/0/1396847821.jpg',
 'gs://cassava_gao/raw/test/0/1400687003.jpg',
 'gs://cassava_gao/raw/test/0/1410898675.jpg',
 'gs://cassava_

In [ ]:
class TF_Augment(tf.keras.layers.Layer):
    def __init__(self,
                resize = None, ## (height, width)
                rescale = None,  ## Factor
                normalize = False,
                random_flip = None, #  "horizontal", "vertical", or "horizontal_and_vertical"
                contrast = None, # factor or tuple (low_bound, high_bound)
                rotation = None, # tuple as lower and upper bound in percentage of a complete rotation
                translation = None, # table of tuple, (low, up) in percentage, the first is for vertical shift, the second for horizontal shift
                zoom = None, # tuple lower bound upper bound 
                ):
        super(TF_Augment, self).__init__()
        self.processor = []
        
        if resize is not None:
            self.processor.append(tf.keras.layers.experimental.preprocessing.Resizing(resize[0], resize[1]))
            
        if rescale is not None:
            self.processor.append(tf.keras.layers.experimental.preprocessing.Rescaling(rescale))
        
        if normalize:
            self.processor.append(tf.keras.layers.experimental.preprocessing.Normalization())
        
        if random_flip is not None:
            self.processor.append(tf.keras.layers.experimental.preprocessing.RandomFlip(random_flip))
            
        if contrast is not None:
            self.processor.append(tf.keras.layers.experimental.preprocessing.RandomContrast(contrast))
        
        if rotation is not None:
            self.processor.append(tf.keras.layers.experimental.preprocessing.RandomRotation(rotation))
        
        if translation is not None:
            self.processor.append(tf.keras.layers.experimental.preprocessing.RandomTranslation(translation[0], translation[1]))
        
        if zoom is not None:
            self.processor.append(tf.keras.layers.experimental.preprocessing.RandomZoom(zoom[0], zoom[1]))
            
    def call(self,x, training = True):
        
        for elt in self.processor:
            x = elt(x, training = training)
        
        return x

In [ ]:
inputs = tf.keras.Input(shape = (128,128,3))

x = hub.KerasLayer("https://tfhub.dev/tensorflow/efficientnet/b7/classification/1", trainable = False)(inputs)

# x = hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_50/classification/4", trainable = True)(inputs)
# x = hub.KerasLayer("https://tfhub.dev/google/inaturalist/inception_v3/feature_vector/4",trainable=True)(inputs)

x = tf.keras.layers.Dense(5, activation = 'softmax')(x)

model = tf.keras.Model(inputs, x)

In [ ]:
model.load_weights('./checkpoint/b7.h5')
# model.load_weights('./checkpoint/efficient/weights')

In [ ]:
model.summary()

In [ ]:
img_size = 128
batch_size = 64

aug = TF_Augment(resize = None, ## (height, width)
                rescale = None,  ## Factor
                normalize = False,
                random_flip = 'horizontal_and_vertical', #  "horizontal", "vertical", or "horizontal_and_vertical"
                contrast = (0,0.1), # factor or tuple (low_bound, high_bound)
                rotation = (-0.1,0.1), # tuple as lower and upper bound in percentage of a complete rotation
                translation = ((-0.1,0.1),(-0.1,0.1)), # table of tuple, (low, up) in percentage, the first is for vertical shift, the second for horizontal shift
                zoom = ((-0.1,0.1),(-0.1,0.1)), # tuple lower bound upper bound )
            )

def decode_image(img):
    image = tf.image.decode_jpeg(img, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.reshape(image, [512,512, 3])
    image = tf.image.resize(image, [img_size,img_size])
    
    image = aug(tf.expand_dims(image, axis = 0), training = True)[0]
    
    return image

def parse_img(example, labeled=True):
    FEATURES = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.int64)
    } if labeled else {
        "image": tf.io.FixedLenFeature([], tf.string),
        "image_name": tf.io.FixedLenFeature([], tf.string)
    }
    example_img = tf.io.parse_single_example(example, FEATURES)
    image = decode_image(example_img['image'])
    if labeled:
        label = tf.cast(example_img['target'], tf.int32)
        return image, label
    idnum = example_img['image_name']
    return image, idnum

base_folder = './train_tfrecords'
# base_folder = 'gs://cassava_gao/tf_record'


train_filenames = base_folder + "/ld_train*-1338.tfrec"
test_filenames = base_folder + "/ld_train*-1327.tfrec"

def get_dataset( gcs_pattern, batch_size = 256):
    list_data = tf.io.gfile.glob(gcs_pattern)
    dataset = tf.data.TFRecordDataset(list_data)
    dataset = dataset.map(parse_img, num_parallel_calls=8)
    dataset = dataset.repeat().batch(batch_size)
    return dataset.prefetch(tf.data.experimental.AUTOTUNE)

train_dataset = get_dataset(train_filenames, batch_size = batch_size)
test_dataset = get_dataset(test_filenames, batch_size = batch_size)

In [ ]:
# from vit_keras import vit, utils
# image_size = 512
# model = vit.vit_b32(
#     image_size=image_size,
#     activation='softmax',
#     pretrained=True,
#     include_top=True,
#     pretrained_top=False,
#     classes=5, 
#     weights="imagenet21k+imagenet2012",
# )

In [ ]:
# # !git clone https://github.com/durbin-164/BiTemperedLogisticLossTensorflow.git
# import sys
# sys.path.append('./BiTemperedLogisticLossTensorflow')
# # import all our functions
# from bi_tempered_loss import bi_tempered_logistic_loss

# #For BiTempered loss function
# T_1 = 0.3
# T_2 = 1.2
# SMOOTH_FRACTION = 0.05
# N_ITER = 5

# class BiTemperedLogisticLoss(tf.keras.losses.Loss):
#     def __init__(self, t1, t2, lbl_smth, n_iter):
#         super(BiTemperedLogisticLoss, self).__init__()
#         self.t1 = t1
#         self.t2 = t2
#         self.lbl_smth = lbl_smth
#         self.n_iter = n_iter

#     def call(self, y_true, y_pred):
#         return bi_tempered_logistic_loss(y_pred, y_true, self.t1, self.t2, self.lbl_smth, self.n_iter)

In [ ]:
import tensorflow_addons as tfa
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
model.compile(
#               loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
              loss = 'sparse_categorical_crossentropy',
#               loss = BiTemperedLogisticLoss(t1=T_1, t2=T_2, lbl_smth=SMOOTH_FRACTION, n_iter=N_ITER),
#              optimizer = SGD(0.001),
#              optimizer = RMSprop(0.01),
             optimizer = Adam(2e-5),
#             optimizer = tfa.optimizers.AdamW(learning_rate=1e-5, weight_decay=1e-4),
             metrics = ['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

early = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=8, verbose=1, 
                                                mode='min', restore_best_weights=True)

reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, 
                           mode='min', min_delta=0.0001, cooldown=0, min_lr=0)

callbacks =[early, reduce]

epochs = 100
steps_per_epoch = int(21000/batch_size)
validation_steps = int(1400/batch_size)

# tg = train_gen()

# history = model.fit(train_datagen, validation_data = test_datagen, epochs = epochs, 
#                    steps_per_epoch = steps_per_epoch, validation_steps = validation_steps,
#                    callbacks = callbacks)

history = model.fit(train_dataset, validation_data = test_dataset, epochs = epochs, 
                   steps_per_epoch = steps_per_epoch, validation_steps = validation_steps,
                   callbacks = callbacks)

In [ ]:
model.save('./checkpoint/b7.h5')

In [6]:
import numpy as np
import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="./ggautomodel/tf_lite/model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)

# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[{'name': 'image', 'index': 0, 'shape': array([  1, 224, 224,   3]), 'shape_signature': array([  1, 224, 224,   3]), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.007874015718698502, 128), 'quantization_parameters': {'scales': array([0.00787402], dtype=float32), 'zero_points': array([128]), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


ValueError: Cannot set tensor: Got value of type FLOAT32 but expected type UINT8 for input 0, name: image 

In [ ]:
inception v3 baseline 128*128 : val_acc = 0.7716
efficientnet B0 Baseline 128*128 : val_acc = 
resnet 50 v2 Baseline 128*128 : val_acc = 

In [3]:
import tensorflow as tf
export_path = './automodel_saved_model.pb'
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess, ['serve'], export_path)
    
path = './autovision/6103.jpg'
with open(path, "rb") as img_file:
    y_pred = sess.run('tile:0', feed_dict={'normalised_input_image_tensor': [img_file.read()] })
    
print(y_pred)

AttributeError: module 'tensorflow' has no attribute 'Session'

In [1]:
import tensorflow as tf
model = tf.saved_model.load(
    './ggautomodel/automodel_saved_model.pb', tags=None, options=None
)

OSError: SavedModel file does not exist at: ./ggautomodel/automodel_saved_model.pb/{saved_model.pbtxt|saved_model.pb}

In [ ]:
class TF_Augment(tf.keras.layers.Layer):
    def __init__(self,
                resize = None, ## (height, width)
                rescale = None,  ## Factor
                normalize = False,
                random_flip = None, #  "horizontal", "vertical", or "horizontal_and_vertical"
                contrast = None, # factor or tuple (low_bound, high_bound)
                rotation = None, # tuple as lower and upper bound in percentage of a complete rotation
                translation = None, # table of tuple, (low, up) in percentage, the first is for vertical shift, the second for horizontal shift
                zoom = None, # tuple lower bound upper bound 
                ):
        super(TF_Augment, self).__init__()
        self.processor = []
        
        if resize is not None:
            self.processor.append(tf.keras.layers.experimental.preprocessing.Resizing(resize[0], resize[1]))
            
        if rescale is not None:
            self.processor.append(tf.keras.layers.experimental.preprocessing.Rescaling(rescale))
        
        if normalize:
            self.processor.append(tf.keras.layers.experimental.preprocessing.Normalization())
        
        if random_flip is not None:
            self.processor.append(tf.keras.layers.experimental.preprocessing.RandomFlip(random_flip))
            
        if contrast is not None:
            self.processor.append(tf.keras.layers.experimental.preprocessing.RandomContrast(contrast))
        
        if rotation is not None:
            self.processor.append(tf.keras.layers.experimental.preprocessing.RandomRotation(rotation))
        
        if translation is not None:
            self.processor.append(tf.keras.layers.experimental.preprocessing.RandomTranslation(translation[0], translation[1]))
        
        if zoom is not None:
            self.processor.append(tf.keras.layers.experimental.preprocessing.RandomZoom(zoom))
            
    def call(self,x, training = True):
        
        for elt in self.processor:
            x = elt(x, training = training)
        
        return x
            
        

In [ ]:
aug = TF_Augment(resize = (128,128), ## (height, width)
                rescale = 1/255,  ## Factor
                normalize = False,
                random_flip = 'horizontal', #  "horizontal", "vertical", or "horizontal_and_vertical"
                contrast = None, # factor or tuple (low_bound, high_bound)
                rotation = None, # tuple as lower and upper bound in percentage of a complete rotation
                translation = None, # table of tuple, (low, up) in percentage, the first is for vertical shift, the second for horizontal shift
                zoom = (-0.1,0.1), # tuple lower bound upper bound )
            )

In [ ]:
input_img = np.random.random((32, 224, 224, 3)).astype('float32')

In [ ]:
a = aug(input_img)

In [ ]:
#  model.save_weights('./checkpoint/inception/weights')
model.save_weights('./checkpoint/weights.h5')

In [ ]:
c = 0
Y = []
pred = []
for x, y in test_datagen:
    if c >= 100:
        break
    else:
        p = model.predict(x)
        Y.append(y)
        pred.append(p)
        
    print(c)
    c+=1

In [ ]:
Y = np.concatenate(Y)
pred = np.concatenate(pred)

In [ ]:
Y1 = np.argmax(Y, axis = -1)
pred1 = np.argmax(pred, axis = -1)

In [ ]:
(pred1 == Y1).sum()/len(Y1)